In [13]:
import os
import tensorflow            as tf
import numpy                 as np
import matplotlib.pyplot     as plt
import tensorflow_datasets   as tfds
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Lambda, Dropout

In [20]:
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-f9b3d719-9ffe-0dab-1700-db0fab379a6f)


In [2]:
(train_set, val_set), info = tfds.load('cats_vs_dogs',
                                        split         = ['train[:80%]', 'train[80%:]'],
                                        as_supervised = True,
                                        with_info     = True)

2024-11-02 12:42:41.218300: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
IMG_SIZE   = (224,224)
BATCH_SIZE = 256

In [4]:
def preprocess_image(image, label):
    image  = tf.image.resize(image, IMG_SIZE)
    image  = tf.cast(image, tf.float32) / 255.0
    return image, label

In [5]:
training_set   = train_set.map(preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)
training_set   = training_set.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

validation_set = val_set.map(preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)
validation_set = validation_set.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [6]:
val_size       = int(0.4 * len(validation_set))
test_set       = validation_set.take(val_size)
validation_set = validation_set.skip(val_size)

In [7]:
images, labels = next(iter(training_set))

image = images[0]
print(f'Image Shape: {image.shape}\nImage dtype: {image.dtype}\nMax pixel value: {np.max(image)}')

Image Shape: (224, 224, 3)
Image dtype: <dtype: 'float32'>
Max pixel value: 1.0


In [18]:
import random
from tensorflow.keras.metrics      import Precision, Recall, AUC
from tensorflow.keras.backend      import epsilon
from tensorflow.keras.regularizers import l2

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [9]:
def set_random_seeds(seed_value):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)

In [16]:
def create_model():
    base_model = tf.keras.applications.DenseNet121(include_top = False,
                                                   input_shape = (224,224,3),
                                                   weights     = 'imagenet',)
    base_model.trainable = False

    x       = base_model.output
    x       = GlobalAveragePooling2D()(x)
    x       = Dropout(0.2)(x)
    x       = Dense(512,
                    activation         ='relu',
                    kernel_regularizer = l2(0.001))(x)
    x       = Dropout(0.4)(x)
    outputs = Dense(1,
                    activation         ='sigmoid',
                    kernel_regularizer = l2(0.001))(x)

    model   = tf.keras.Model(inputs = base_model.input, outputs=outputs)
    
    
    model.compile(optimizer = tf.keras.optimizers.Adam(),
                 loss       = 'binary_crossentropy',
                 metrics    = ['accuracy',
                         Precision(name = 'precision'),
                         Recall(name    = 'recall'),
                         AUC(name       = 'auc'),
                         F1Score(name   = 'f1_score')])
    return model

In [23]:
num_trials  = 10
results     = []

# Create directory if it doesn't exist
save_dir    = "models/diff_seeds"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for i in range(num_trials):
    print(f"Trial {i+1}")
    seed    = i  # Different seed for each trial
    set_random_seeds(seed)

    model   = create_model()
    
    history = model.fit(training_set, validation_data=validation_set, epochs=10, verbose=2)

    val_acc = history.history['val_accuracy'][1]
    print(f"Validation Accuracy for trial {i+1}: {val_acc}")
    
    results.append({
        'trial'   : i+1,
        'seed'    : seed,
        'val_acc' : val_acc,
        'history' : history.history
    })

    test_loss, test_acc, _, _, _, _ = model.evaluate(test_set)
    print(f"Test Accuracy for trial {i+1}: {test_acc}")
    
    model_name = f'DenseNet{i+1}.h5'
    model_path = os.path.join(save_dir, model_name)

    # Save the model if both validation and test accuracy are greater than 0.95
    if val_acc > 0.95 and test_acc > 0.95:
        model.save(model_path)
        print(f"Model saved as {model_path}")

Trial 1
Epoch 1/10
73/73 - 74s - loss: 0.5077 - accuracy: 0.9546 - precision: 0.9516 - recall: 0.9571 - auc: 0.9927 - f1_score: 0.9544 - val_loss: 0.3065 - val_accuracy: 0.9843 - val_precision: 0.9861 - val_recall: 0.9827 - val_auc: 0.9989 - val_f1_score: 0.9844 - 74s/epoch - 1s/step
Epoch 2/10
73/73 - 67s - loss: 0.2707 - accuracy: 0.9809 - precision: 0.9807 - recall: 0.9807 - auc: 0.9980 - f1_score: 0.9807 - val_loss: 0.2375 - val_accuracy: 0.9780 - val_precision: 0.9915 - val_recall: 0.9647 - val_auc: 0.9990 - val_f1_score: 0.9779 - 67s/epoch - 916ms/step
Epoch 3/10
73/73 - 67s - loss: 0.2058 - accuracy: 0.9833 - precision: 0.9830 - recall: 0.9834 - auc: 0.9984 - f1_score: 0.9832 - val_loss: 0.1844 - val_accuracy: 0.9822 - val_precision: 0.9901 - val_recall: 0.9744 - val_auc: 0.9991 - val_f1_score: 0.9822 - 67s/epoch - 911ms/step
Epoch 4/10
73/73 - 67s - loss: 0.1677 - accuracy: 0.9839 - precision: 0.9830 - recall: 0.9845 - auc: 0.9984 - f1_score: 0.9838 - val_loss: 0.1587 - val_acc

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as models/diff_seeds/DenseNet1.h5
Trial 2
Epoch 1/10
73/73 - 73s - loss: 0.5461 - accuracy: 0.9462 - precision: 0.9436 - recall: 0.9481 - auc: 0.9899 - f1_score: 0.9459 - val_loss: 0.3271 - val_accuracy: 0.9815 - val_precision: 0.9827 - val_recall: 0.9806 - val_auc: 0.9987 - val_f1_score: 0.9816 - 73s/epoch - 1s/step
Epoch 2/10
73/73 - 67s - loss: 0.2907 - accuracy: 0.9821 - precision: 0.9816 - recall: 0.9822 - auc: 0.9984 - f1_score: 0.9819 - val_loss: 0.2497 - val_accuracy: 0.9832 - val_precision: 0.9854 - val_recall: 0.9813 - val_auc: 0.9990 - val_f1_score: 0.9833 - 67s/epoch - 919ms/step
Epoch 3/10
73/73 - 66s - loss: 0.2311 - accuracy: 0.9821 - precision: 0.9823 - recall: 0.9816 - auc: 0.9982 - f1_score: 0.9820 - val_loss: 0.2077 - val_accuracy: 0.9804 - val_precision: 0.9922 - val_recall: 0.9688 - val_auc: 0.9991 - val_f1_score: 0.9804 - 66s/epoch - 906ms/step
Epoch 4/10
73/73 - 66s - loss: 0.1849 - accuracy: 0.9846 - precision: 0.9847 - recall: 0.9843 - auc: 0.9986 -